In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

First define the quadrotor parameters




In [ ]:
# Quadrotor Parameters and Transfer Functions

# Import required libraries
import numpy as np
import control as ctrl
import matplotlib.pyplot as plt
from control.matlab import step, margin, rlocus


# Define quadrotor parameters
m = 0.5  # kg - Mass of quadcopter
ep = 10 * np.pi / 180  # radians - Dihedral angle
g = 9.81  # m/s^2 - Gravitational acceleration
l = 0.5  # m - Length of drone
J = (1 / 12) * m * l**2  # kg*m^2 - Moment of inertia (cylinder)

# Define transfer functions
s = ctrl.TransferFunction.s  # Define Laplace variable 's'

G_tu = (m * g * l * np.cos(ep)) / (J * s**2)  # Transfer function for torque-to-angular displacement
G_xt = ((-m * g * l) - (np.tan(ep) * J * s**2)) / (s**2 * m * l)  # Transfer function for displacement-to-torque

# Display the transfer functions
print("Transfer Function G_tu (Torque-to-Angular Displacement):")
print(G_tu)

print("\nTransfer Function G_xt (Displacement-to-Torque):")
print(G_xt)


c) Design for C_uΘ

In [ ]:
# Plot Bode plot for G_tu
plt.figure()

# Generate the Bode plot
ctrl.bode_plot(G_tu, dB=True, Hz=False, deg=True)


In [ ]:
# Compute margins for G_tu
GM_tu, PM_tu, wgc_tu, wpc_tu = ctrl.margin(G_tu)

# Display the results
print(f"Gain Margin (GM): {GM_tu} dB")
print(f"Phase Margin (PM): {PM_tu} degrees")
print(f"Gain Crossover Frequency (wgc): {wgc_tu} rad/s")
print(f"Phase Crossover Frequency (wpc): {wpc_tu} rad/s")


There are many ways to go about the design that we have seen in the lecture, such as dynamic compensators (and then inferring the PID gains), or the root locus design method, pole-zero placement, ... We leave it open for you to play around with this and choose gains according to what you would deem acceptable performance.


In [ ]:
# Going by adding two zeros to attract the poles to the left (PID as two zeroes, an integrator and a fast pole). Providing this only as
#example design (no specific performance criteria, just providing template)
z_1 = TODO
z_2 = TODO
p_fast = TODO
C_tu = (s+z_1)*(s+z_2)/(s*(s+p_fast));
ctrl.root_locus(G_tu*C_tu)

In [ ]:
k_tu = 50; # adjusted to max phase margin from bode plot
L_tu = k_tu*G_tu*C_tu;
# generate bode plot of loop gain with controller
ctrl.bode(L_tu);


In [ ]:
# Compute margins for L_tu

[GM_Ltu,PM_Ltu,wgc_Ltu,wpc_Ltu]=ctrl.margin(L_tu)

# Display the results
print(f"Gain Margin (GM): {GM_Ltu} dB")
print(f"Phase Margin (PM): {PM_Ltu} degrees")
print(f"Gain Crossover Frequency (wgc): {wgc_Ltu} rad/s")
print(f"Phase Crossover Frequency (wpc): {wpc_Ltu} rad/s")

In [ ]:
# Calculate closed-loop TF and generate step response
T_tu = L_tu/(1+L_tu)

time, response = ctrl.step_response(T_tu, T=2)

# Plot the step response

plt.figure()
plt.plot(time, response)
plt.title('Step Response of T_tu')
plt.xlabel('Time (s)')
plt.ylabel('Response')
plt.xlim([0,2])
plt.grid(True)
plt.show()


In [ ]:
# Gather step response info
ctrl.step_info(response, time)

d) Following similar steps as above. Again, there are multiple ways to solve this. We leave this open on purpose. Here it is important however to note that the bandwidth of the outer loop should be significantly smaller than the inner loop. A common rule of thumb is to make it 10 times slower, which means that we could/should at most consider a bandwidth of around 3 rad/s! (Since the inner loop has a bandwidth of approx 30 rad/s)

In [ ]:
ctrl.bode(G_xt)

In [ ]:
# Compute margins for G_xt
[GM_tx,PM_tx,wgc_tx,wpc_tx]=ctrl.margin(G_xt)

# Display the results
print(f"Gain Margin (GM): {GM_tx} dB")
print(f"Phase Margin (PM): {PM_tx} degrees")
print(f"Gain Crossover Frequency (wgc): {wgc_tx} rad/s")
print(f"Phase Crossover Frequency (wpc): {wpc_tx} rad/s")


In [ ]:
# define controller transfer function
p_fast_2 = TODO
p_slow = TODO
z_3 = TODO
C_xt = (s+z_3)/((s+p_slow)*(s+p_fast_2));
ctrl.root_locus(-G_xt*C_xt);

In [ ]:
# generate loop transfer function
k_xt = TODO; # adjusted to max phase margin from bode plot and to not exceed
L_xt = k_xt*G_xt*C_xt;
ctrl.bode(L_xt);

In [ ]:
# compute margins for L_xt
[GM_Lxt,PM_Lxt,wgc_Lxt,wpc_Lxt]=ctrl.margin(L_xt)

# Display the results
print(f"Gain Margin (GM): {GM_Lxt} dB")
print(f"Phase Margin (PM): {PM_Lxt} degrees")
print(f"Gain Crossover Frequency (wgc): {wgc_Lxt} rad/s")
print(f"Phase Crossover Frequency (wpc): {wpc_Lxt} rad/s")

In [ ]:
# Calculate closed-loop TF and generate step response
T_xt = L_xt/(1+L_xt)
time2, response2 = ctrl.step_response(T_xt, T=5)

# Plot the step response

plt.figure()
plt.plot(time2, response2)
plt.title('Step Response of T_xt')
plt.xlabel('Time (s)')
plt.ylabel('Response')
plt.xlim([0,5])
plt.grid(True)
plt.show()

In [ ]:
# Gather step response info
ctrl.step_info(response2, time2)

In [ ]:
# check closed loop bandwidth
ctrl.bode(T_xt)